In [67]:
import numpy as np
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import math
import talib as ta

In [68]:
#We're going to load data from the QQQ ETF, which tracks the top 100 non-financial companies listed on the Nasdaq
#This will be from the beginning of 2022 to present.

QQQ = yf.download('QQQ', start = '2022-01-01', end = '2024-01-06')
QQQ

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,399.049988,401.940002,396.880005,401.679993,396.005310,40575900
2022-01-04,402.239990,402.279999,393.290009,396.470001,390.868896,58027200
2022-01-05,394.739990,395.890015,384.029999,384.290009,378.861023,75739800
2022-01-06,382.420013,387.350006,380.130005,384.019989,378.594849,70814300
2022-01-07,384.029999,385.760010,378.040009,379.859985,374.493530,72652300
...,...,...,...,...,...,...
2023-12-29,411.279999,411.640015,407.579987,409.519989,409.519989,42633400
2024-01-02,405.839996,406.089996,400.239990,402.589996,402.589996,58026900
2024-01-03,399.929993,401.000000,397.890015,398.329987,398.329987,47002800


In [69]:
QQQ = QQQ.drop(columns=['Adj Close'])
QQQ

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03,399.049988,401.940002,396.880005,401.679993,40575900
2022-01-04,402.239990,402.279999,393.290009,396.470001,58027200
2022-01-05,394.739990,395.890015,384.029999,384.290009,75739800
2022-01-06,382.420013,387.350006,380.130005,384.019989,70814300
2022-01-07,384.029999,385.760010,378.040009,379.859985,72652300
...,...,...,...,...,...
2023-12-29,411.279999,411.640015,407.579987,409.519989,42633400
2024-01-02,405.839996,406.089996,400.239990,402.589996,58026900
2024-01-03,399.929993,401.000000,397.890015,398.329987,47002800


In [70]:
#We're going to be using the backtesting.py framework which allows us to backest trading and investment strategies
#First we're going to do DCA for Dollar Cost Averaging

class DCA(Strategy):
    
    amount_to_invest = 10
    
    #we're going to be putting our indiciators in the init function as we only need to calculate them once
    
    def init(self):
        
        #we're going to work out the day of the week so we know what day to invest
        #in this framework self.data.Close is given as a numpy array, we're going to convert to series so we can get date values
        #we're calculating an indicator, using an empty function x to start
    
        self.day_of_week = self.I(lambda x: x, self.data.Close.s.index.dayofweek)
       
    
    #this acts as the main function to which we implement our trading logic
    
    def next(self):
        
        #0 is Monday, Friday is 4, if current day of week is Tuesday, we buy
        if self.day_of_week[-1] == 1:
            
            #need whole numbers for amounts of shares to buy so using math.floor to round to nearest whole number
            
            self.buy(size = math.floor(self.amount_to_invest / self.data.Close[-1])) 

#going to make share price smaller so we have more sensible share sizing
QQQ = QQQ * 10**-6

bt = Backtest(QQQ, DCA, trade_on_close = True)
stats = bt.run()

trades = stats["_trades"]

print(trades)

price_paid = trades["Size"] * trades["EntryPrice"]

total_invested = price_paid.sum()

current_shares = trades["Size"].sum()
current_equity = current_shares * QQQ.Close.iloc[-1]

print("Total Investment:", total_invested)
print("Current Shares:", current_shares / (10**6))
print("Current Equity:", current_equity)
print("Return Percentage:", ((current_equity/total_invested)-1)*100)
bt.plot()

C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(


      Size  EntryBar  ExitBar  EntryPrice  ExitPrice       PnL  ReturnPct  \
0    24839       501      503    0.000403   0.000396 -0.156734  -0.015674   
1    24338       497      503    0.000411   0.000396 -0.355335  -0.035534   
2    24440       493      503    0.000409   0.000396 -0.314787  -0.031479   
3    25083       488      503    0.000399   0.000396 -0.059949  -0.005995   
4    25820       483      503    0.000387   0.000396  0.232122   0.023213   
..     ...       ...      ...         ...        ...       ...        ...   
99   27358        20      503    0.000366   0.000396  0.841532   0.084154   
100  28976        15      503    0.000345   0.000396  1.482702   0.148272   
101  26986        10      503    0.000371   0.000396  0.694350   0.069437   
102  25918         6      503    0.000386   0.000396  0.271102   0.027111   
103  25222         1      503    0.000396   0.000396 -0.004792  -0.000479   

     EntryTime   ExitTime Duration  
0   2024-01-02 2024-01-04   2 days  
1

C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p7073', ...)

In [71]:
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,24839,501,503,0.000403,0.000396,-0.156734,-0.015674,2024-01-02,2024-01-04,2 days
1,24338,497,503,0.000411,0.000396,-0.355335,-0.035534,2023-12-26,2024-01-04,9 days
2,24440,493,503,0.000409,0.000396,-0.314787,-0.031479,2023-12-19,2024-01-04,16 days
3,25083,488,503,0.000399,0.000396,-0.059949,-0.005995,2023-12-12,2024-01-04,23 days
4,25820,483,503,0.000387,0.000396,0.232122,0.023213,2023-12-05,2024-01-04,30 days
...,...,...,...,...,...,...,...,...,...,...
99,27358,20,503,0.000366,0.000396,0.841532,0.084154,2022-02-01,2024-01-04,702 days
100,28976,15,503,0.000345,0.000396,1.482702,0.148272,2022-01-25,2024-01-04,709 days
101,26986,10,503,0.000371,0.000396,0.694350,0.069437,2022-01-18,2024-01-04,716 days
102,25918,6,503,0.000386,0.000396,0.271102,0.027111,2022-01-11,2024-01-04,723 days


In [80]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.daily_sma1 = self.I(ta.SMA, self.data.Close, self.n1)
        self.daily_sma2 = self.I(ta.SMA, self.data.Close, self.n2)

        self.weekly_sma1 = resample_apply("W", ta.SMA, self.data.Close, self.n1)
        self.weekly_sma2 = resample_apply("W", ta.SMA, self.data.Close, self.n2)

    
    
    def next(self):
        # If weekly sma1 crosses above weekly sma2, we will buy
        if crossover(self.weekly_sma1, self.weekly_sma2):
            self.buy()

        # Else, if weekly sma1 crosses below weekly sma2, close any existing
        # long trades
        elif crossover(self.weekly_sma2, self.weekly_sma1):
            self.position.close()
               
bt = Backtest(QQQ, SmaCross, cash=1040)
stats = bt.run()
print(stats)

trades = stats["_trades"]
print(trades)

price_paid = trades["Size"] * trades["EntryPrice"]
print(price_paid)

bt.plot()

print("Final Percentage Return:", stats.iloc[6])


Start                     2022-01-03 00:00:00
End                       2024-01-05 00:00:00
Duration                    732 days 00:00:00
Exposure Time [%]                   48.712871
Equity Final [$]                  1252.071016
Equity Peak [$]                   1299.601977
Return [%]                          20.391444
Buy & Hold Return [%]               -1.227343
Return (Ann.) [%]                    9.702876
Volatility (Ann.) [%]               15.881339
Sharpe Ratio                         0.610961
Sortino Ratio                        0.993133
Calmar Ratio                         0.603014
Max. Drawdown [%]                  -16.090621
Avg. Drawdown [%]                   -3.055498
Max. Drawdown Duration      200 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      29.211266
Worst Trade [%]                     -9.660244
Avg. Trade [%]                    

C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\jj_do\anaconda3\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Final Percentage Return: 20.391443820706893
